<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèles mis en oeuvre : 
    * Multinomial Naive Bayes
    * Multinomial Naive Bayes chaîné
    * Regression logistic (GSCV)
    * SVM / SGD (GSCV)
    * Multinomial Bernoulli 

Ce notebook utilise les données issues des notebooks : 


**`P6_DadaAnalysis.ipynb`**

**`P6_UnsupervizedMethods.ipynb`**



In [43]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# <font color=blus>0. Data loading</font>

In [30]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV

import p6_util

def gscv_best_classifier_result(dict_param_grid, classifier, X_train, y_train\
                              , cls_name, X_test,y_test, dict_cls_score):
    ovr_classifier = OneVsRestClassifier(classifier)
    gscv_classifier  = GridSearchCV(ovr_classifier, dict_param_grid)

    gscv_classifier.fit(X_train, y_train)
    print (gscv_classifier.best_score_)
    print (gscv_classifier.best_params_)
    y_pred = gscv_classifier.best_estimator_.predict(X_test)
    cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
    print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))
    dict_cls_score[cls_name]=cls_score
    return dict_cls_score, gscv_classifier.best_estimator_

    

In [2]:
import pandas as pd

import p5_util
import p6_util

is_reloaded = True

if is_reloaded is True :
    file_name="./data/vectorizer_tfidf_ngram1_b.dump"
    file_name="./data/vectorizer_tfidf_ngram1_0.0001_.dump"
    vectorizer = p5_util.object_load(file_name)


    file_name="./data/csr_matrix_tfidf_ngram1_b.dump"
    file_name="./data/csr_matrix_tfidf_ngram1_0.0001_.dump"
    csr_matrix_tfidf_1gram_train = p5_util.object_load(file_name)
    print(csr_matrix_tfidf_1gram_train.shape)

    file_name='./data/csr_matrix_tdif_1gram_test.dump'
    csr_matrix_tdif_1gram_test = p5_util.object_load(file_name)
    print(csr_matrix_tdif_1gram_test.shape)

    
    
    file_name="./data/csr_matrix_encoded_train_tags.dump"
    csr_matrix_encoded_train_tags = p5_util.object_load(file_name)
    print(csr_matrix_encoded_train_tags.shape)

    file_name="./data/csr_matrix_encoded_test_tags.dump"
    csr_matrix_encoded_test_tags = p5_util.object_load(file_name)
    print(csr_matrix_encoded_test_tags.shape)
    
    # Processing test dataset from vectorizer and tokenized dataset.
    file_name='./data/ser_sof_body_test.dump'
    ser_sof_body_test = p5_util.object_load(file_name)

    csr_matrix_tdif_1gram_test = vectorizer.transform(ser_sof_body_test)
    print(csr_matrix_tdif_1gram_test.shape)
    
else :
    file_name="./data/vectorizer_tfidf_ngram1_b.dump"
    vectorizer = p5_util.object_load(file_name)

    file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
    ser_sof_tags_train=pd.read_csv(file_name)['Tags']
    print(ser_sof_tags_train.shape)

    file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
    ser_sof_tags_test=pd.read_csv(file_name)['Tags']
    print(ser_sof_tags_test.shape)

    #-------------------------------------------------------------------------------
    # String of TAGs are converted as list of TAGS.
    #  * Multiple TAGs are assigned to a unique POST.
    # This lead to deal with a multi-label classification issue.
    # All labels (TAGs) are one hot encoded.
    #  * Train and test data-sets do not have same assigned TAGs set.
    # This leads to a different number of encoded columns for each data-set.
    # For having same number of columns for train and test datasets, 
    # encoding process uses the union of train and test assigned TAGs .
    #-------------------------------------------------------------------------------

    # List of referenced TAG is built...
    list_list_tags_train  \
    = ser_sof_tags_train.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
    print(len(list_list_tags))
    
    list_list_tags_test \
    = ser_sof_tags_test.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
    print(len(list_list_tags))
    
    list_ref_tags = list()
    for list_tags in list_list_tags_train+list_list_tags_test :
        list_ref_tags += [tag for tag in list_tags]

    list_ref_tags = list(set(list_ref_tags))
    print(len(list_ref_tags))
    #-------------------------------------------------------------------------------
    
    

    # Assigned TAG from train dataset are encoded 
    csr_matrix_encoded_train_tags, list_ref_train_tags \
    = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_train, list_ref_tags)
    print("csr_matrix_encoded_train_tags : {}".format(csr_matrix_encoded_train_tags.shape))

    file_name="./data/csr_matrix_encoded_train_tags.dump"
    p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)
    
    # Assigned TAG from test dataset are encoded 
    csr_matrix_encoded_test_tags, list_ref_test_tags \
    = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_test, list_ref_tags)
    print("csr_matrix_encoded_test_tags : {}".format(csr_matrix_encoded_test_tags.shape))

    file_name="./data/csr_matrix_encoded_test_tags.dump"
    p5_util.object_dump(csr_matrix_encoded_test_tags,file_name)
    #-------------------------------------------------------------------------------
    

    #-------------------------------------------------------------------------------
    # Test dataset is standardized.
    #-------------------------------------------------------------------------------
    file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
    df_sof_test=pd.read_csv(file_name)
    print(df_sof_test.shape)

    df_sof_test['Body']=df_sof_test['Body']+df_sof_test['Title']
    ser_sof_body_test = p6_util.p6_df_standardization(df_sof_test.Body)

    file_name='./data/ser_sof_body_test.dump'
    p5_util.object_dump(ser_sof_body_test, file_name)

    csr_matrix_tdif_1gram_test = vectorizer.transform(ser_sof_body_test)
    file_name='./data/csr_matrix_tdif_1gram_test.dump'
    p5_util.object_dump(csr_matrix_tdif_1gram_test,file_name)
    print(csr_matrix_tdif_1gram_test.shape)


file_name="./data/csr_matrix_tdif_1gram_test.dump"
csr_matrix_tfidf_1gram_test = p5_util.object_load(file_name)

X_train = csr_matrix_tfidf_1gram_train
y_train = csr_matrix_encoded_train_tags

X_test = csr_matrix_tdif_1gram_test
y_test = csr_matrix_encoded_test_tags

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

p5_util.object_load : fileName= ./data/vectorizer_tfidf_ngram1_0.0001_.dump
p5_util.object_load : fileName= ./data/csr_matrix_tfidf_ngram1_0.0001_.dump
(24604, 9037)
p5_util.object_load : fileName= ./data/csr_matrix_tdif_1gram_test.dump
(16359, 6448)
p5_util.object_load : fileName= ./data/csr_matrix_encoded_train_tags.dump
(24604, 6436)
p5_util.object_load : fileName= ./data/csr_matrix_encoded_test_tags.dump
(16359, 6436)
p5_util.object_load : fileName= ./data/ser_sof_body_test.dump
(16359, 9037)
p5_util.object_load : fileName= ./data/csr_matrix_tdif_1gram_test.dump
(24604, 9037) (24604, 6436) (16359, 9037) (16359, 6436)


In [3]:
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_test=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_test.shape)

ser_sof_tags_test = ser_sof_tags_test.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>')

(16359,)


In [5]:
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_train=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_train.shape)


# List of referenced TAG is built...
list_list_tags_train  \
= ser_sof_tags_train.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags_train))

list_unique_tags_train = list()
for list_tags_train in list_list_tags_train:
    list_unique_tags_train += [tag for tag in list_tags_train]
list_unique_tags_train = list(set(list_unique_tags_train))
print(len(list_unique_tags_train))



(24604,)
24604
6436


Removing rows from test dataset with TAG that are not into list of TAG from train dataset

In [12]:
list_row_keep = list()
ser_sof_test_restricted=  pd.Series()
print(len(ser_sof_tags_test))
# TAGS from test dataset that are not into list of TAG from train dataset are removed
for row, list_tag_row in ser_sof_tags_test.items():
    is_inserted = True
    for tag_row in list_tag_row :
        if tag_row not in list_unique_tags_train :
            is_inserted = False
            break
        else: 
            pass
    if is_inserted is True :
        list_row_keep.append(row)
    else :
        #print(list_tag_row)
        pass
print(len(list_row_keep)/len(ser_sof_tags_test))
print(len(list_row_keep))

X_test = X_test[list_row_keep]
ser_sof_tags_test = ser_sof_tags_test.iloc[list_row_keep]
X_test.shape, ser_sof_tags_test.shape

y_test = y_test[list_row_keep]
y_test.shape

X_train = X_train[list_row_keep]
X_train.shape

y_train = y_train[list_row_keep]
y_train.shape

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14161, 9037), (14161, 9037), (14161, 6436), (14161, 6436))

In [22]:
len(list_unique_tags_train)

6436

In [23]:
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_train=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_train.shape)

# Assigned TAG from train dataset are encoded 
csr_matrix_encoded_train_tags, list_ref_train_tags \
= p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_train, list_unique_tags_train)
print("csr_matrix_encoded_train_tags : {}".format(csr_matrix_encoded_train_tags.shape))

file_name="./data/csr_matrix_encoded_train_tags.dump"
p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)

file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_test=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_test.shape)

# Assigned TAG from test dataset are encoded 
csr_matrix_encoded_test_tags, list_ref_test_tags \
= p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_test, list_unique_tags_train)
print("csr_matrix_encoded_test_tags : {}".format(csr_matrix_encoded_test_tags.shape))

file_name="./data/csr_matrix_encoded_test_tags.dump"
p5_util.object_dump(csr_matrix_encoded_test_tags,file_name)

y_train = csr_matrix_encoded_train_tags

y_test = csr_matrix_encoded_test_tags

(24604,)
csr_matrix_encoded_train_tags : (24604, 6436)
(16359,)
csr_matrix_encoded_test_tags : (16359, 6436)


In [24]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(24604, 9037) (24604, 6436) (14161, 9037) (16359, 6436)


# <font color=blus>4. Supervized methods</font>

## <font color=blus>4.1. Bernoulli Naive Bayes </font>

In [14]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
N_JOBS=-1
cls_ber = OneVsRestClassifier(BernoulliNB(alpha=5.e-2), n_jobs=N_JOBS).fit(X_train, y_train)

(14161, 9037) (14161, 9037) (14161, 6436) (14161, 6436)


**Compute accuracy**

In [15]:
y_pred = cls_ber.predict(X_test)
y_pred.shape


(14161, 6436)

In [16]:
import p6_util
import p5_util
cls_name = 'Bernoulli NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred,mode_match='intersection_matching'\
                                                     ,encoder=None, arr_encoded_filter=None)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)
dict_cls_score[cls_name] = cls_score
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score


Mean accuracy score for Bernoulli NB : 0.70 %
p5_util.object_load : fileName= ./data/dict_cls_score.dump


{'SGD': 0.001206950369368432,
 'Linear Regression': 0.0010612839454791384,
 'Logistic Regression': 0.009551555509312247,
 'SGD Classifier': 0.0,
 'Bernoulli NB': 0.00701363073110285,
 'Chained Multinomial NB': 7.434944237918216e-05,
 'Multinomial NB': 7.434944237918216e-05,
 'SVM/SGD': 0.0,
 'MNB': 7.06164818868724e-05}

## <font color=blus>4.2. Applying Naive Bayes classifier</font>

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes. 

Then, one classifier per class is fitted.



In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

cls_mnb = OneVsRestClassifier(MultinomialNB(alpha=5e-2), n_jobs=-1).fit(X_train, y_train)
cls_mnb

OneVsRestClassifier(estimator=MultinomialNB(alpha=0.05, class_prior=None, fit_prior=True),
          n_jobs=-1)

**TAGs predictions and probabilities for test dataset**

In [18]:
y_pred = cls_mnb.predict(X_test)
y_pred.shape

(14161, 6436)

In [19]:
import p5_util
import p6_util

file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

cls_name = 'Multinomial NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score))
cls_score

dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

p5_util.object_load : fileName= ./data/dict_cls_score.dump
Accuracy score for Multinomial NB : 0.00 %


{'SGD': 0.001206950369368432,
 'Linear Regression': 0.0010612839454791384,
 'Logistic Regression': 0.009551555509312247,
 'SGD Classifier': 0.0,
 'Bernoulli NB': 0.00701363073110285,
 'Chained Multinomial NB': 7.434944237918216e-05,
 'Multinomial NB': 0.000322180916976456,
 'SVM/SGD': 0.0,
 'MNB': 7.06164818868724e-05}

**Compute accuracy**

In [52]:
y_test.shape, y_pred.shape

((16359, 6436), (14161, 6436))

In [59]:
from sklearn.metrics import accuracy_score
import p5_util

y_pred = cls_mnb.predict(X_test)

cls_mnb_score = accuracy_score(y_test  ,y_pred)

print("Accuracy score for MNB classifier : {0:1.2F} %".format(cls_mnb_score*100))

dict_cls_score['MNB'] = cls_mnb_score
dict_cls_score


file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score,file_name)


Accuracy score for MNB classifier : 0.01 %


## <font color='blus'>4.3. Chained Multinomial Naive Bayes classifier</font>

In [60]:
import p5_util
import p6_util

cls_name = 'Chained Multinomial NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


file_name="./data/dict_cls_score.dump"
dict_cls_score = p5_util.object_load(file_name)
dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

Mean accuracy score for Chained Multinomial NB : 0.01 %
p5_util.object_load : fileName= ./data/dict_cls_score.dump


{'SGD': 0.001206950369368432,
 'Linear Regression': 0.0010612839454791384,
 'Logistic Regression': 0.009551555509312247,
 'SGD Classifier': 0.0,
 'Bernoulli NB': 0.0036926889714993805,
 'Chained Multinomial NB': 7.434944237918216e-05,
 'Multinomial NB': 7.434944237918216e-05,
 'SVM/SGD': 0.0,
 'MNB': 7.06164818868724e-05}

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import BernoulliNB

chained_classifier_ber = ClassifierChain(BernoulliNB(alpha=5.e-2))


chained_classifier_ber.fit(X_train , y_train)

y_pred = chained_classifier_ber.predict(X_test)

In [ ]:
import p5_util
import p6_util

cls_name = 'Chained Bernoulli NB'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


file_name="./data/dict_cls_score.dump"
dict_cls_score = p5_util.object_load(file_name)
dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

## <font color='blus'>4.4. SVM/SGD classifier</font>

In [ ]:
import p5_util

dict_param_grid = {
    # 'estimator__alpha': [ 1e-3,1e-2,1e-1], : ne donne rien
    #'estimator__alpha': [ 1.0,5.0,10.,20.,50.], : meilleur classifieur : 1.; ne donne rien
    #'estimator__alpha': [ 0.5,0.6,0.7,0.8,0.9,1.0,1.1], : meilleur classifieur : 0.5; ne donne rien
    'estimator__alpha': [ 0.2,0.3,0.4,], 
    'estimator__loss': [ "hinge"],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}
cls_name='SVM/SGD'
dict_cls_score, best_classfier = gscv_best_classifier_result(dict_param_grid,SGDClassifier(), X_train, y_train\
                                               , cls_name, X_test, y_test, dict_cls_score)
file_name = './data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_best_SVM.dump'
p5_util.object_dump(best_classfier, file_name)
dict_cls_score

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

ovr_sgd_classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)).fit(X_train, y_train)


print (ovr_sgd_classifier)


In [ ]:
y_pred = ovr_sgd_classifier.predict(X_test)

In [ ]:
import p6_util
import p5_util

cls_name = 'SVM/SGD'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


file_name="./data/dict_cls_score.dump"
dict_cls_score = p5_util.object_load(file_name)
dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

## <font color='blus'>4.5. Grid Search CV Logistic regression & One vs rest</font>

In [20]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

dict_param_grid = {
    'estimator__C': [ 1e-3, 1e-2, 1e-1, 1e0, 5, 10],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}

estimator = OneVsRestClassifier(LogisticRegression())
gscv_lr_classifier  = GridSearchCV(estimator, dict_param_grid)

gscv_lr_classifier.fit(X_train, y_train)

print (gscv_lr_classifier.best_score_)
print (gscv_lr_classifier.best_params_)

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 149 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 150 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 152 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 153 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 156 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 157 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 274 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 275 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 276 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 279 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 282 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 283 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 416 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 417 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 418 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 423 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 427 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 432 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 562 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 563 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 564 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 568 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 571 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 574 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5349 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5354 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5358 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5363 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5364 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5366 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5649 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5654 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5655 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5676 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5695 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5697 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6023 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6032 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6043 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6044 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6054 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6056 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6364 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6377 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6394 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6401 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6407 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6410 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

0.0
{'estimator__C': 0.001, 'estimator__penalty': 'l2', 'n_jobs': -1}


In [23]:
import p6_util
gscv_lr_classifier.best_estimator_
y_pred = gscv_lr_classifier.best_estimator_.predict(X_test)
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))
cls_score


Mean accuracy score for Multinomial NB : 0.00 %


0.0

In [ ]:
from sklearn.linear_model import LogisticRegression

import p5_util
import p6_util

logreg_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2', C=10, n_jobs=-1))

logreg_classifier.fit(X_train, y_train)

y_pred = logreg_classifier.predict(X_test)

In [ ]:
cls_name = 'Logistic Regression'
cls_score = p6_util.p6_supervized_mean_accuracy_score(y_test, y_pred)
print("Mean accuracy score for "+cls_name+" : {0:1.2F} %".format(cls_score*100))


dict_cls_score[cls_name] = cls_score

file_name='./data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)
dict_cls_score

In [ ]:
LogisticRegression()

In [33]:
import p5_util

dict_param_grid = {
    #'estimator__C': [ 150,200],#0.011350681536555142,
    #'estimator__C': [ 0.01, 0.1,1.], #0.0
    'estimator__C': [ 100,150],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}
cls_name='Logistic Regression'
dict_cls_score, best_classfier = gscv_best_classifier_result(dict_param_grid,LogisticRegression(), X_train, y_train\
                                               , cls_name, X_test, y_test, dict_cls_score)
file_name = './data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_best_logisticRegression.dump'
p5_util.object_dump(best_classfier, file_name)
dict_cls_score

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.0007061648188687239
{'estimator__C': 150, 'estimator__penalty': 'l2', 'n_jobs': -1}
Mean accuracy score for Logistic Regression : 0.99 %


{'SGD': 0.001206950369368432,
 'Linear Regression': 0.0010612839454791384,
 'Logistic Regression': 0.009888475836431227,
 'SGD Classifier': 0.0,
 'Bernoulli NB': 0.00701363073110285,
 'Chained Multinomial NB': 7.434944237918216e-05,
 'Multinomial NB': 0.000322180916976456,
 'SVM/SGD': 0.0,
 'MNB': 7.06164818868724e-05}

In [ ]:
import p5_util
file_name = './data/dict_cls_score.dump'
p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_best_logisticRegression.dump'
p5_util.object_dump(best_classfier, file_name)


In [ ]:
file_name = './data/classifier_best_logisticRegression.dump'
best_classfier= p5_util.object_load(file_name)

In [ ]:
best_classfier

In [25]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import parfit.parfit as pf

def parfit_best_classifier_result(dict_param_grid, classifier, X_train, y_train\
                              , cls_name, X_test, y_test, dict_cls_score):
    paramGrid = ParameterGrid(dict_param_grid)
    
    bestModel, bestScore, allModels, allScores = pf.bestFit(classifier, paramGrid,\
           X_train, y_train, X_test, y_test, metric = roc_auc_score, \
           scoreLabel = "AUC")
    
    print(bestModel, bestScore)
    return bestModel


In [26]:
X_train.shape, y_train.shape

((14161, 9037), (14161, 6436))

## <font color='blus'>4.6. Grid search for SVM classifier with SGD</font>

In [ ]:
from sklearn.linear_model import SGDClassifier
import p5_util
file_name = './data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_param_grid = {
    'estimator__loss':['hinge'],
    #'estimator__alpha': [1e-3, 5*1e-3, 1e-2, 5e-2, 1e-1,5e-1,1.,10,20],
    'estimator__alpha': [ 1e-1],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}

cls_name='SGD Classifier'
dict_cls_score, best_classfier = gscv_best_classifier_result(dict_param_grid, SGDClassifier(), X_train, y_train\
                                               , cls_name, X_test, y_test,dict_cls_score)


p5_util.object_dump(dict_cls_score, file_name)

file_name = './data/classifier_bestSGD_SVM.dump'
p5_util.object_dump(best_classfier, file_name)
dict_cls_score

## <font color='blus'>4.7. Powered SGD classifier</font>

In [ ]:
from sklearn.linear_model import SGDClassifier
from skmultilearn.problem_transform import LabelPowerset
power_classifier_sgd = LabelPowerset(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))
power_classifier_sgd.fit(X_train, y_train)

y_pred = power_classifier_sgd.predict(X_test)

power_classifier_sgd_score = accuracy_score(y_test[:,:col_min]  ,y_pred[:,:col_min])

print("Accuracy mean score for Powered SGD classifier : {0:1.2F} %".format(power_classifier_sgd_score*100))

dict_cls_score['Powered SGD'] = power_classifier_sgd_score

In [ ]:
import p5_util
file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_cls_score['GSCV SGD Regression'] = gscv_sgd_cls.best_score_
p5_util.object_dump(dict_cls_score, file_name)

## <font color='blus'>4.8. Gaussian Naive Bayes classifier</font>

# <font color='blus'>5. Classifiers results</font>

In [ ]:
import pandas as pd
df_result = pd.DataFrame.from_dict( dict_cls_score, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result

In [ ]:
import p6_util_plot
title = "Classifiers accuracy"
p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False)

In [ ]:
dict_benchmark_result = dict()
dict_benchmark_result['TFIDF(1,2)']=8.0/100
dict_benchmark_result['BOW']=8.5/100
dict_benchmark_result['TFIDF(1,1)']=12/100
dict_benchmark_result['Kmeans']=16/100
dict_benchmark_result['LDA-TFIDF']=22/100
dict_benchmark_result['LDA-BOW']=30/100
dict_benchmark_result['W2VEC']=24/100
dict_benchmark_result['MNB']=2/100
dict_benchmark_result['SGD']=4/100
dict_benchmark_result['Bernoulli NB']=20/100
dict_benchmark_result['Logistic Reg.']=25/100
dict_benchmark_result['Chained MNB']=0./100



In [ ]:
import pandas as pd
df_result = pd.DataFrame.from_dict( dict_benchmark_result, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result

In [ ]:
import p6_util_plot
title = "Classifiers accuracy"
p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title,\
                                    p_reverse=False)